In [3]:
import matplotlib.pyplot as plt
import librosa
import numpy as np
import pandas as pd
import os
import IPython.display as ipd
from IPython.display import clear_output
%matplotlib inline

In [4]:
root_path="C:/Users/sarib/Desktop/Gohm/SpeechRecognition/database"

In [5]:
def mfcc_extractor(file):
    signal, sample_rate = librosa.load(file)
    ipd.Audio(file)
    mfccs_features = librosa.feature.mfcc(y=signal, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T, axis=0)
    
    return mfccs_scaled_features

In [6]:
#NOT USED
def fft_extractor(file):
    signal, sample_rate = librosa.load(file)
    ipd.Audio(file)
    fft = np.fft.fft(signal)
    
    return fft

In [7]:
#NOT USED
signal, sample_rate = librosa.load('SpeechRecognition/database/ac/ac_AHCG_FSRPNCW.wav')
ipd.Audio('database/ac/ac_AHCG_FSRPNCW.wav')
fft = np.fft.fft(signal)
fft

array([ 3.96959355+0.j        , -1.57368256-0.43014124j,
       -0.5562213 +0.51962332j, ...,  0.00995858+0.12657352j,
       -0.5562213 -0.51962332j, -1.57368256+0.43014124j])

In [8]:
#NOT USED
signal, sample_rate = librosa.load('SpeechRecognition/database\sol\sol_AKZI_FAWCVUT.wav' )
ipd.Audio('SpeechRecognition/database\sol\sol_AKZI_FAWCVUT.wav' )
mfccs_features = librosa.feature.mfcc(y=signal, sr=sample_rate, n_mfcc=40)
mfccs_features.shape

(40, 44)

In [9]:
"""from tqdm import tqdm
import time"""
extracted_feataures =[]
for root, directories, files in os.walk(root_path):
    for file_names in files:
        if root is not root_path:
            #print(os.path.join(root,file_names))
            data = mfcc_extractor(os.path.join(root,file_names))
            extracted_feataures.append([data,os.path.basename(os.path.normpath(root))])

    """for i in tqdm (range (100), desc="Loading..."):
        time.sleep(3)"""

In [10]:
extracted_feataures_dataframe= pd.DataFrame(extracted_feataures, columns=["mfcc", "word"])
extracted_feataures_dataframe.head()

,mfcc,word
0,"[-483.0338, 62.193104, -10.424236, 17.89255, -...",ac
1,"[-393.0876, 61.467396, -20.944004, 18.326431, ...",ac
2,"[-557.86304, 99.75818, -6.094904, 0.65864086, ...",ac
3,"[-518.9073, 52.873825, -3.625966, 5.807297, -9...",ac
4,"[-539.5348, 84.21058, -6.7271714, 4.8634925, -...",ac


In [11]:
X=np.array(extracted_feataures_dataframe['mfcc'].tolist())
Y=np.array(extracted_feataures_dataframe['word'].tolist())

In [12]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
Y_numerical=to_categorical(labelencoder.fit_transform(Y))

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test= train_test_split(X,Y_numerical, test_size=0.2, random_state=0)

In [14]:
#NOT USED
X_train.shape

(21153, 40)

In [15]:
#NOT USED
len(Y_test)

5289

ANN Model

In [16]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping,CSVLogger
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [17]:
model= Sequential()

model.add(Flatten(input_shape=(40, )))
model.add(Dense(300, Activation('relu')))
model.add(Dropout(0.2))
model.add(Dense(300, Activation('relu')))
model.add(Dense(Y_numerical.shape[1], Activation('softmax')))

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 40)                0         
                                                                 
 dense (Dense)               (None, 300)               12300     
                                                                 
 dropout (Dropout)           (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 300)               90300     
                                                                 
 dense_2 (Dense)             (None, 14)                4214      
                                                                 
Total params: 106,814
Trainable params: 106,814
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer=Adam())

In [20]:
#SAVING WITH CHECKPOINTS
"""checkpointpath="training3/cp.ckpt"
checkpointer = ModelCheckpoint(checkpointpath, monitor='val_accuracy',
                               verbose=2, save_best_only=True, mode='max')

early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=2)
callbackList= [checkpointer, early_stop]
model.fit(X_train, Y_train, epochs=200, validation_data=(X_test, Y_test), callbacks=callbackList, verbose=2)"""



'checkpointpath="training3/cp.ckpt"\ncheckpointer = ModelCheckpoint(checkpointpath, monitor=\'val_accuracy\',\n                               verbose=2, save_best_only=True, mode=\'max\')\n\nearly_stop = EarlyStopping(monitor=\'val_loss\', patience=10, verbose=2)\ncallbackList= [checkpointer, early_stop]\nmodel.fit(X_train, Y_train, epochs=200, validation_data=(X_test, Y_test), callbacks=callbackList, verbose=2)'

In [37]:
#SAVING THE WHOLE MODEL
checkpointpath="ANN/trainingset/cp.ckpt"
#checkpointpath="C:/Users/sarib/Desktop/Gohm/SpeechRecognition/ANN/saved_models/weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpointer = ModelCheckpoint(checkpointpath, monitor='val_accuracy',
                               verbose=2, save_best_only=True, mode='max')

#early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=2)
callbackList= [checkpointer]
pretrained_model=model.fit(X_train, Y_train, epochs=200, validation_data=(X_test, Y_test), callbacks=callbackList, verbose=2)
#model.save('ANN/completesavedModel/')

Epoch 1/200

Epoch 1: val_accuracy improved from -inf to 0.76082, saving model to trainingset\cp.ckpt


INFO:tensorflow:Assets written to: trainingset\cp.ckpt\assets


INFO:tensorflow:Assets written to: trainingset\cp.ckpt\assets


662/662 - 4s - loss: 0.3524 - accuracy: 0.8849 - val_loss: 1.0091 - val_accuracy: 0.7608 - 4s/epoch - 6ms/step
Epoch 2/200

Epoch 2: val_accuracy improved from 0.76082 to 0.76687, saving model to trainingset\cp.ckpt


INFO:tensorflow:Assets written to: trainingset\cp.ckpt\assets


...

INFO:tensorflow:Assets written to: trainingset\cp.ckpt\assets


662/662 - 4s - loss: 0.3578 - accuracy: 0.8820 - val_loss: 0.9529 - val_accuracy: 0.7674 - 4s/epoch - 6ms/step
Epoch 4/200

Epoch 4: val_accuracy did not improve from 0.76744
662/662 - 2s - loss: 0.3562 - accuracy: 0.8830 - val_loss: 0.9854 - val_accuracy: 0.7591 - 2s/epoch - 4ms/step
Epoch 5/200

Epoch 5: val_accuracy did not improve from 0.76744
662/662 - 3s - loss: 0.3647 - accuracy: 0.8797 - val_loss: 0.9846 - val_accuracy: 0.7533 - 3s/epoch - 4ms/step
Epoch 6/200

Epoch 6: val_accuracy did not improve from 0.76744
662/662 - 2s - loss: 0.3606 - accuracy: 0.8833 - val_loss: 1.0210 - val_accuracy: 0.7555 - 2s/epoch - 3ms/step
Epoch 7/200

Epoch 7: val_accuracy did not improve from 0.76744
662/662 - 2s - loss: 0.3594 - accuracy: 0.8821 - val_loss: 0.9951 - val_accuracy: 0.7567 - 2s/epoch - 3ms/step
Epoch 8/200

Epoch 8: val_accuracy did not improve from 0.76744
662/662 - 2s - loss: 0.3781 - accuracy: 0.8777 - val_loss: 0.9840 - val_accuracy: 0.7574 - 2s/epoch - 4ms/step
Epoch 9/200

E

INFO:tensorflow:Assets written to: trainingset\cp.ckpt\assets


INFO:tensorflow:Assets written to: trainingset\cp.ckpt\assets


662/662 - 3s - loss: 0.3437 - accuracy: 0.8886 - val_loss: 1.0047 - val_accuracy: 0.7705 - 3s/epoch - 4ms/step
Epoch 78/200

Epoch 78: val_accuracy did not improve from 0.77047
662/662 - 2s - loss: 0.3517 - accuracy: 0.8864 - val_loss: 1.0394 - val_accuracy: 0.7538 - 2s/epoch - 4ms/step
Epoch 79/200

Epoch 79: val_accuracy did not improve from 0.77047
662/662 - 2s - loss: 0.3474 - accuracy: 0.8905 - val_loss: 1.0056 - val_accuracy: 0.7623 - 2s/epoch - 3ms/step
Epoch 80/200

Epoch 80: val_accuracy did not improve from 0.77047
662/662 - 2s - loss: 0.3392 - accuracy: 0.8912 - val_loss: 1.0100 - val_accuracy: 0.7599 - 2s/epoch - 3ms/step
Epoch 81/200

Epoch 81: val_accuracy did not improve from 0.77047
662/662 - 2s - loss: 0.3285 - accuracy: 0.8938 - val_loss: 1.0513 - val_accuracy: 0.7525 - 2s/epoch - 3ms/step
Epoch 82/200

Epoch 82: val_accuracy did not improve from 0.77047
662/662 - 2s - loss: 0.3496 - accuracy: 0.8892 - val_loss: 1.0082 - val_accuracy: 0.7597 - 2s/epoch - 3ms/step
Epoc

In [38]:
print("Test Score: ", model.evaluate(X_test,Y_test))
print("Train Score: ", model.evaluate(X_train,Y_train))

166/166 [==============================] - 0s 2ms/step - loss: 1.0924 - accuracy: 0.7546
Test Score:  [1.0923938751220703, 0.7545849680900574]
662/662 [==============================] - 1s 2ms/step - loss: 0.0813 - accuracy: 0.9748
Train Score:  [0.08133947849273682, 0.9748498797416687]


In [52]:
"""plt.plot(model.history.history['loss'], label='train')
plt.plot(model.history.history['val_loss'], label='test')
plt.legend()
plt.show()"""

"plt.plot(model.history.history['loss'], label='train')\nplt.plot(model.history.history['val_loss'], label='test')\nplt.legend()\nplt.show()"

In [56]:
from keras.models import load_model
#pretrained_model = load_model('completesavedModel/')
model.load_weights(checkpointpath)

In [41]:
print("Test Score: ", model.evaluate(X_test,Y_test))
print("Train Score: ", model.evaluate(X_train,Y_train))

166/166 [==============================] - 0s 2ms/step - loss: 1.0047 - accuracy: 0.7705
Test Score:  [1.004654884338379, 0.7704669833183289]
662/662 [==============================] - 2s 2ms/step - loss: 0.0801 - accuracy: 0.9766
Train Score:  [0.0801207572221756, 0.976646363735199]


PREDICTION WITH DATA

In [43]:
predictiondata_feataures =[]
counter_true=0
for root, directories, files in os.walk("C:/Users/sarib/Desktop/Gohm/SpeechRecognition/RecordsFromDataset"): 
    for file_names in files:
        if root is not root_path:
            #print(os.path.join(root,file_names) )
            r = mfcc_extractor(os.path.join(root,file_names))
            rl=r.reshape(1,-1)
            predicted_label=np.argmax(model.predict(rl), axis=-1)
            prediction_class = labelencoder.inverse_transform(predicted_label) 
            predictiondata_feataures.append([os.path.basename(os.path.normpath(root)), prediction_class, predicted_label])
            if os.path.basename(os.path.normpath(root))==prediction_class:
                counter_true+=1

In [44]:
predictiondata_feataures_dataframe= pd.DataFrame(predictiondata_feataures, columns=["actual data", "predicted data", "predicted data code"])
percentage= (counter_true/len(predictiondata_feataures))*100
print("The percantage of the true predicted data: ", percentage,"%")
#predictiondata_feataures_dataframe.to_csv("output.csv")
predictiondata_feataures_dataframe


The percantage of the true predicted data:  66.66666666666666 %


,actual data,predicted data,predicted data code
0,ac,[ac],[0]
1,ac,[dur],[4]
2,ac,[ac],[0]
3,asagi,[asagi],[1]
4,asagi,[hayir],[7]
5,asagi,[asagi],[1]
6,baslat,[baslat],[2]
7,baslat,[hayir],[7]
8,baslat,[asagi],[1]
9,devam,[devam],[3]


In [47]:
predictiondatas_feataures =[]
counters_true=0
for root, directories, files in os.walk("C:/Users/sarib/Desktop/Gohm/SpeechRecognition/Records"): 
    for file_names in files:
        if root is not root_path:
            #print(os.path.join(root,file_names) )
            r = mfcc_extractor(os.path.join(root,file_names))
            rl=r.reshape(1,-1)
            predicteds_label=np.argmax(model.predict(rl), axis=-1)
            predictions_class = labelencoder.inverse_transform(predicteds_label) 
            predictiondatas_feataures.append([os.path.basename(os.path.normpath(root)), predictions_class, predicteds_label])
            if os.path.basename(os.path.normpath(root))==predictions_class:
                counters_true+=1

In [48]:
predictiondatas_feataures_dataframe= pd.DataFrame(predictiondatas_feataures, columns=["actual data", "predicted data", "predicted data code"])
percentages= (counters_true/len(predictiondatas_feataures))*100
print("The percantage of the true predicted data: ", percentages,"%")
#predictiondata_feataures_dataframe.to_csv("output.csv")
predictiondatas_feataures_dataframe


The percantage of the true predicted data:  47.61904761904761 %


,actual data,predicted data,predicted data code
0,ac,[ac],[0]
1,ac,[ac],[0]
2,ac,[asagi],[1]
3,asagi,[asagi],[1]
4,asagi,[asagi],[1]
5,asagi,[asagi],[1]
6,baslat,[baslat],[2]
7,baslat,[baslat],[2]
8,baslat,[baslat],[2]
9,devam,[sag],[11]


In [ ]:
"""filename="predictionData/dursultan.wav"
r=mfcc_extractor(filename)
print(r)
rl=r.reshape(1,-1)
print(rl)
print(rl.shape)
predicted_label=np.argmax(model.predict(rl), axis=-1)

print(predicted_label)
prediction_class = labelencoder.inverse_transform(predicted_label) 
prediction_class"""

'filename="predictionData/dursultan.wav"\nr=mfcc_extractor(filename)\nprint(r)\nrl=r.reshape(1,-1)\nprint(rl)\nprint(rl.shape)\npredicted_label=np.argmax(model.predict(rl), axis=-1)\n\nprint(predicted_label)\nprediction_class = labelencoder.inverse_transform(predicted_label) \nprediction_class'